# Analyzing HHS hospital occupancy data

#### An analysis of HHS hospital data, by @datagraphics and @stiles.

#### **Questions?** [matt.stiles@latimes.com](matt.stiles@latimes.com) \\ 310.529.8749

---

### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt

%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat

alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

### Define cities and regions metadata

In [3]:
metadata_df = pd.read_csv("input/city-metadata.csv", dtype={"fips": str})

In [4]:
metadata_df = metadata_df[["county", "fips", "population", "region"]]

In [5]:
socal = ["Los Angeles", "Orange", "Ventura", "San Bernardino", "Riverside"]
bayarea = [
    "Alameda",
    "Contra Costa",
    "Marin",
    "Napa",
    "San Francisco",
    "San Mateo",
    "Santa Clara",
    "Solano",
    "Sonoma",
]

### Get the latest url from the HHS API and read in the latest dataframe

In [6]:
# Data dictionary: https://healthdata.gov/covid-19-reported-patient-impact-and-hospital-capacity-facility-data-dictionary
# Data source: https://healthdata.gov/dataset/covid-19-reported-patient-impact-and-hospital-capacity-facility

In [7]:
src = pd.read_csv(
    "https://healthdata.gov/api/views/anag-cw7u/rows.csv?accessType=DOWNLOAD",
    dtype={"fips_code": str, "zip": str},
    infer_datetime_format=True,
    parse_dates=True,
)

In [8]:
# src = pd.read_csv('https://healthdata.gov/sites/default/files/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20201228.csv',\
# dtype={'fips_code':str, 'zip':str}, infer_datetime_format=True, parse_dates=True)

In [9]:
df = src.copy()

In [10]:
ca_data = pd.DataFrame(df.loc[df.state == "CA"])

In [11]:
ca_data["fips"] = ca_data["fips_code"].str[2:5]

In [12]:
ca_timeseries = pd.merge(
    ca_data, metadata_df, how="left", left_on="fips", right_on="fips"
)

### What's the most recent collection week?

In [13]:
ca_timeseries["collection_week"] = pd.to_datetime(ca_timeseries["collection_week"])

In [14]:
ca_timeseries["collection_week"].max()

Timestamp('2021-03-19 00:00:00')

### Get rid of the -999999 suppressed values

In [15]:
ca_timeseries = ca_timeseries.replace([-999999.0], [0])

In [16]:
ca_timeseries["hospital_name"] = (ca_timeseries["hospital_name"]).str.title()
ca_timeseries["hospital_name"] = (
    (ca_timeseries["hospital_name"])
    .str.replace(" Of ", " of ")
    .str.replace("Hlth", "Health")
    .str.replace(" La", " LA")
)

---

## Calculate some hospitalization/covid rates

In [17]:
ca_timeseries.head()

hospital_pk collection_week state     ccn  \
0      050002      2021-03-19    CA  050002   
1      050195      2021-03-19    CA  050195   
2      050320      2021-03-19    CA  050320   
3      050488      2021-03-19    CA  050488   
4      050075      2021-03-19    CA  050075   

                                   hospital_name  \
0                               St Rose Hospital   
1                            Washington Hospital   
2                              Highland Hospital   
3                            Eden Medical Center   
4  Kaiser Foundation Hospital - Oakland/Richmond   

                        address           city    zip hospital_subtype  \
0            27200 CALAROGA AVE        HAYWARD  94545       Short Term   
1                2000 MOWRY AVE        FREMONT  94538       Short Term   
2         1411 EAST 31ST STREET        OAKLAND  94602       Short Term   
3        20103 LAKE CHABOT ROAD  CASTRO VALLEY  94546       Short Term   
4  275 WEST MACARTHUR BOULEVARD        OAKLAND  94611       Short Term   

  fips_code  is_metro_micro  total_beds_7_day_avg  \
0     06001            True                  95.0   
1     06001            True                 215.0   
2     06001            True                 432.0   
3     06001            True                 150.1   
4     06001            True                 313.0   

   all_adult_hospital_beds_7_day_avg  \
0                               95.0   
1                              202.0   
2                              424.0   
3                              150.0   
4                              223.9   

   all_adult_hospital_inpatient_beds_7_day_avg  inpatient_beds_used_7_day_avg  \
0                                         95.0                           53.3   
1                                        180.0                          159.9   
2                                        424.0                          157.9   
3                                        132.6                          126.3   
4                                        223.9                          243.1   

   all_adult_hospital_inpatient_bed_occupied_7_day_avg  \
0                                               53.3     
1                                              157.4     
2                                              156.7     
3                                              125.9     
4                                              203.1     

   total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg  \
0                                                0.0                           
1                                                6.9                           
2                                               10.7                           
3                                               11.6                           
4                                               26.0                           

   total_adult_patients_hospitalized_confirmed_covid_7_day_avg  \
0                                                0.0             
1                                                5.6             
2                                                9.3             
3                                               10.4             
4                                               18.6             

   total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg  \
0                                                0.0                               
1                                                0.0                               
2                                                0.0                               
3                                                0.0                               
4                                                0.0                               

   total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg  \
0                                                0.0                 
1                                         

#### How many beds

In [18]:
ca_timeseries["total_beds"] = round(
    (
        ca_timeseries["total_beds_7_day_sum"]
        / ca_timeseries["total_beds_7_day_coverage"]
    ),
    0,
)

#### Calculate daily average of total staffed ICU beds

In [19]:
ca_timeseries["total_staffed_adult_icu_beds"] = round(
    (
        ca_timeseries["total_staffed_adult_icu_beds_7_day_sum"]
        / ca_timeseries["total_staffed_adult_icu_beds_7_day_coverage"]
    ),
    0,
)

#### Calculate daily average of occupied ICU beds

In [20]:
ca_timeseries["total_occupied_adult_icu_beds"] = round(
    (
        ca_timeseries["staffed_adult_icu_bed_occupancy_7_day_sum"]
        / ca_timeseries["staffed_adult_icu_bed_occupancy_7_day_coverage"]
    ),
    0,
)

#### Calculate daily average of COVID patients in the ICU

In [21]:
ca_timeseries["total_covid_icu_patients"] = round(
    (
        ca_timeseries[
            "staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum"
        ]
        / ca_timeseries[
            "staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_coverage"
        ]
    ),
    0,
)

#### Number of ICU beds available

In [22]:
ca_timeseries["total_available_adult_icu_beds"] = round(
    (
        ca_timeseries["total_staffed_adult_icu_beds"]
        - ca_timeseries["total_occupied_adult_icu_beds"]
    ),
    0,
)

#### Calculate daily ICU occupancy as percentage

In [23]:
ca_timeseries["pct_occupied_adult_icu_beds"] = round(
    (
        ca_timeseries["total_occupied_adult_icu_beds"]
        / ca_timeseries["total_staffed_adult_icu_beds"]
    ),
    2,
)

#### If a hospital reports 0 staffed adult ICU beds, drop them from the dataframe

In [24]:
filtered_timeseries = ca_timeseries[
    (ca_timeseries.total_staffed_adult_icu_beds_7_day_sum.notnull())
    & (ca_timeseries.total_staffed_adult_icu_beds_7_day_sum != 0)
].sort_values("total_staffed_adult_icu_beds_7_day_sum")

#### How many patients?

In [25]:
ca_timeseries["all_patients"] = round(
    (
        ca_timeseries["inpatient_beds_used_7_day_sum"]
        / ca_timeseries["inpatient_beds_used_7_day_coverage"]
    ),
    0,
)

#### How many Covid patients

In [26]:
ca_timeseries["total_adult_covid_patients"] = (
    ca_timeseries[
        "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum"
    ]
    / ca_timeseries[
        "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_coverage"
    ]
)

In [27]:
ca_timeseries["total_pediatric_covid_patients"] = (
    ca_timeseries[
        "total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum"
    ]
    / ca_timeseries[
        "total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_coverage"
    ]
)

In [28]:
ca_timeseries["total_covid_patients"] = round(
    ca_timeseries["total_adult_covid_patients"]
    + ca_timeseries["total_pediatric_covid_patients"],
    0,
).fillna(0)

#### What's the rate of Covid patients?

In [29]:
ca_timeseries["covid_patients_share"] = round(
    (ca_timeseries["total_covid_patients"] / ca_timeseries["all_patients"]), 2
)

In [30]:
ca_timeseries = ca_timeseries[ca_timeseries["all_patients"] > 0]

---

### Trim to the columns we want

In [31]:
trimmed_timeseries = ca_timeseries[
    [
        "hospital_name",
        "hospital_subtype",
        "ccn",
        "collection_week",
        "county",
        "fips",
        "total_beds",
        "total_staffed_adult_icu_beds",
        "total_covid_icu_patients",
        "total_available_adult_icu_beds",
        "pct_occupied_adult_icu_beds",
        "total_covid_patients",
        "all_patients",
        "covid_patients_share",
        "region",
    ]
].rename(
    columns={
        "hospital_name": "hospital",
        "collection_week": "week",
    }
)

In [32]:
trimmed_timeseries.covid_patients_share.mean()

0.1728466351031367

In [33]:
trimmed_timeseries.to_csv("output/trimmed_timeseries.csv", index=False)

---

### Filter the dataframe for Los Angeles County facilities

In [34]:
la = trimmed_timeseries[trimmed_timeseries["fips"] == "037"]
oc = trimmed_timeseries[trimmed_timeseries["fips"] == "059"]

### Filter the California dataframe to the most recent collection week

In [35]:
current_ca = trimmed_timeseries[
    trimmed_timeseries["week"] == trimmed_timeseries["week"].max()
].sort_values("covid_patients_share", ascending=False)

In [36]:
current_la = la[la["week"] == la["week"].max()].sort_values(
    "covid_patients_share", ascending=False
)

In [37]:
current_oc = oc[oc["week"] == oc["week"].max()].sort_values(
    "covid_patients_share", ascending=False
)

In [38]:
current_ca.to_csv("output/current_ca.csv", index=False)

### Specific place?

In [39]:
current_ca[current_ca["hospital"].str.contains("Methodist Hospital")].iloc[0]

hospital                          Methodist Hospital of Sacramento
hospital_subtype                                        Short Term
ccn                                                         050590
week                                           2021-03-19 00:00:00
county                                                  Sacramento
fips                                                           067
total_beds                                                     206
total_staffed_adult_icu_beds                                    20
total_covid_icu_patients                                         1
total_available_adult_icu_beds                                  13
pct_occupied_adult_icu_beds                                   0.35
total_covid_patients                                             6
all_patients                                                    87
covid_patients_share                                          0.07
region                                          Greater Sacram

### Which CA hospitals have the greatest share of covid patients? 

In [40]:
current_ca[
    (current_ca["hospital_subtype"] != "Long Term")
    & (current_ca["total_covid_icu_patients"] > 0)
][
    [
        "hospital",
        "total_beds",
        "total_covid_patients",
        "all_patients",
        "covid_patients_share",
        "region",
        "county",
    ]
].sort_values(
    "covid_patients_share", ascending=False
).head(
    10
)

hospital  total_beds  \
333                    Oak Valley Hospital District        22.0   
145                       Madera Community Hospital        80.0   
155                     Memorial Hospital Los Banos        41.0   
41                     Mad River Community Hospital        41.0   
277              Kaiser Foundation Hospital Manteca       183.0   
52                Adventist Health Tehachapi Valley        31.0   
340                      Sierra View Medical Center       149.0   
184          Garden Grove Hospital & Medical Center        55.0   
122  Providence Little Co of Mary Med Ctr San Pedro        76.0   
208                  Corona Regional Medical Center       172.0   

     total_covid_patients  all_patients  covid_patients_share  \
333                   5.0          10.0                  0.50   
145                  13.0          30.0                  0.43   
155                   7.0          20.0                  0.35   
41                    5.0          17.0                  0.29   
277                  36.0         157.0                  0.23   
52                    3.0          14.0                  0.21   
340                  13.0          62.0                  0.21   
184                   7.0          33.0                  0.21   
122                  10.0          53.0                  0.19   
208                  20.0         104.0                  0.19   

                  region       county  
333   San Joaquin Valley   Stanislaus  
145   San Joaquin Valley       Madera  
155   San Joaquin Valley       Merced  
41   Northern California     Humboldt  
277   San Joaquin Valley  San Joaquin  
52    San Joaquin Valley         Kern  
340   San Joaquin Valley       Tulare  
184  Southern California       Orange  
122  Southern California  Los Angeles  
208  Southern California    Riverside

### Which facilities in LA have the greatest share of covid patients? 

In [41]:
current_la[
    (current_la["hospital_subtype"] != "Long Term")
    & (current_la["total_covid_icu_patients"] > 0)
][
    [
        "hospital",
        "total_beds",
        "total_covid_patients",
        "all_patients",
        "covid_patients_share",
    ]
].sort_values(
    "covid_patients_share", ascending=False
).head(
    10
)

hospital  total_beds  \
122     Providence Little Co of Mary Med Ctr San Pedro        76.0   
139         Martin Luther King, Jr. Community Hospital       184.0   
137                Cedar-Sinai Marina Del Rey Hospital        91.0   
133               West Hills Hospital & Medical Center       261.0   
71                    Whittier Hospital Medical Center       174.0   
121                             Monterey Park Hospital        96.0   
69   Lac/Rancho Los Amigos National Rehabilitation ...       137.0   
113               Kaiser Foundation Hospital - West LA       219.0   
73                 Kaiser Foundation Hospital - Downey       354.0   
68                                Keck Hospital of Usc       401.0   

     total_covid_patients  all_patients  covid_patients_share  
122                  10.0          53.0                  0.19  
139                  19.0         109.0                  0.17  
137                   8.0          57.0                  0.14  
133                  14.0          98.0                  0.14  
71                   11.0          85.0                  0.13  
121                   6.0          52.0                  0.12  
69                   14.0         118.0                  0.12  
113                  15.0         126.0                  0.12  
73                   21.0         188.0                  0.11  
68                   27.0         245.0                  0.11

### Which places have the lowest share? 

In [42]:
current_la[
    (current_la["hospital_subtype"] != "Long Term")
    & (current_la["total_covid_icu_patients"] > 0)
][
    [
        "hospital",
        "total_beds",
        "total_covid_patients",
        "all_patients",
        "covid_patients_share",
    ]
].sort_values(
    "covid_patients_share", ascending=False
).tail(
    10
)

hospital  total_beds  \
120                           Antelope Valley Hospital       217.0   
138     Providence-Cedars Sinai Tarzana Medical Center       165.0   
83                             Garfield Medical Center       210.0   
134                         Mission Community Hospital       237.0   
111    City of Hope Helford Clinical Research Hospital       231.0   
101                       Pih Health Hospital-Whittier       398.0   
107               Ronald Reagan U C L A Medical Center       460.0   
104  Santa Monica - Ucla Med Ctr & Orthopaedic Hosp...       296.0   
61                              St Mary Medical Center       406.0   
80                         Cedars-Sinai Medical Center       842.0   

     total_covid_patients  all_patients  covid_patients_share  
120                   8.0         215.0                  0.04  
138                   3.0         117.0                  0.03  
83                    4.0         120.0                  0.03  
134                   5.0         153.0                  0.03  
111                   7.0         208.0                  0.03  
101                   7.0         237.0                  0.03  
107                  11.0         389.0                  0.03  
104                   7.0         217.0                  0.03  
61                    5.0         151.0                  0.03  
80                   18.0         749.0                  0.02

### Which facilities in OC have the greatest share of covid patients? 

In [43]:
current_oc[
    (current_oc["hospital_subtype"] != "Long Term")
    & (current_oc["total_covid_icu_patients"] > 0)
][
    ["hospital", "total_covid_patients", "all_patients", "covid_patients_share"]
].sort_values(
    "covid_patients_share", ascending=False
).head(
    5
)

hospital  total_covid_patients  \
184             Garden Grove Hospital & Medical Center                   7.0   
173  Kaiser Foundation Hospital - Orange County - A...                  26.0   
175                     Providence St. Joseph Hospital                  23.0   
178                        Los Alamitos Medical Center                  13.0   
190  Fountain Valley Regional Hospital & Medical Ce...                  14.0   

     all_patients  covid_patients_share  
184          33.0                  0.21  
173         260.0                  0.10  
175         230.0                  0.10  
178         134.0                  0.10  
190         199.0                  0.07

In [44]:
current_oc[
    (current_oc["hospital_subtype"] != "Long Term")
    & (current_oc["total_covid_icu_patients"] > 0)
][
    [
        "hospital",
        "total_beds",
        "total_covid_patients",
        "all_patients",
        "covid_patients_share",
    ]
].sort_values(
    "covid_patients_share", ascending=False
).tail(
    10
)

hospital  total_beds  \
177                       Huntington Beach Hospital        70.0   
193                     Providence Mission Hospital       258.0   
188             Orange County Global Medical Center       337.0   
185                     West Anaheim Medical Center       137.0   
168                   Anaheim Global Medical Center       189.0   
191                La Palma Intercommunity Hospital        39.0   
189                Foothill Regional Medical Center       130.0   
194  University of California Irvine Medical Center       409.0   
182             Hoag Memorial Hospital Presbyterian       395.0   
183          Memorialcare Saddleback Medical Center       228.0   

     total_covid_patients  all_patients  covid_patients_share  
177                   3.0          50.0                  0.06  
193                  10.0         193.0                  0.05  
188                   7.0         183.0                  0.04  
185                   4.0          96.0                  0.04  
168                   5.0         113.0                  0.04  
191                   1.0          27.0                  0.04  
189                   1.0          30.0                  0.03  
194                  10.0         333.0                  0.03  
182                  11.0         355.0                  0.03  
183                   1.0         111.0                  0.01

In [45]:
current_la["covid_patients_share"] = round(
    (current_la["covid_patients_share"] * 100), 2
)

### Output top LA hospitals for CMS table

In [46]:
current_la[
    (current_la["hospital_subtype"] != "Long Term")
    & (current_la["total_covid_icu_patients"] > 0)
][["hospital", "total_covid_patients", "covid_patients_share"]].sort_values(
    "covid_patients_share", ascending=False
).head(
    10
).to_csv(
    "output/current_la_hospitals.csv", index=False
)

### Which facilities in LA have the highest ICU capacity rates? 

In [47]:
current_la[(current_la["total_covid_icu_patients"] > 0)].sort_values(
    "pct_occupied_adult_icu_beds", ascending=False
).head(10)

hospital hospital_subtype     ccn  \
119                      Kindred Hospital South Bay        Long Term  052050   
60            Community Hospital of Huntington Park       Short Term  050091   
138  Providence-Cedars Sinai Tarzana Medical Center       Short Term  050761   
143                      Kindred Hospital Paramount        Long Term  052046   
122  Providence Little Co of Mary Med Ctr San Pedro       Short Term  050078   
144           Glendale Mem Hospital & Health Center       Short Term  050058   
64                     Memorial Hospital of Gardena       Short Term  050468   
120                        Antelope Valley Hospital       Short Term  050056   
102            Providence Holy Cross Medical Center       Short Term  050278   
81                     Henry Mayo Newhall  Hospital       Short Term  050624   

          week       county fips  total_beds  total_staffed_adult_icu_beds  \
119 2021-03-19  Los Angeles  037        59.0                           9.0   
60  2021-03-19  Los Angeles  037        65.0                           6.0   
138 2021-03-19  Los Angeles  037       165.0                           9.0   
143 2021-03-19  Los Angeles  037       177.0                          12.0   
122 2021-03-19  Los Angeles  037        76.0                           9.0   
144 2021-03-19  Los Angeles  037       334.0                          12.0   
64  2021-03-19  Los Angeles  037       172.0                          10.0   
120 2021-03-19  Los Angeles  037       217.0                          30.0   
102 2021-03-19  Los Angeles  037       294.0                          28.0   
81  2021-03-19  Los Angeles  037       378.0                          19.0   

     total_covid_icu_patients  total_available_adult_icu_beds  \
119                       1.0                             0.0   
60                        1.0                             0.0   
138                       1.0                             0.0   
143                       1.0                             0.0   
122                       4.0                             0.0   
144                       2.0                             0.0   
64                        1.0                             0.0   
120                       1.0                             1.0   
102                       8.0                             1.0   
81                        1.0                             1.0   

     pct_occupied_adult_icu_beds  total_covid_patients  all_patients  \
119                         1.00                  11.0          49.0   
60                          1.00                   3.0          30.0   
138                         1.00                   3.0         117.0   
143                         1.00                   3.0          84.0   
122                         1.00                  10.0          53.0   
144                         1.00                   9.0         112.0   
64                          1.00                  10.0         112.0   
120                         0.97                   8.0         215.0   
102                         0.96                  21.0         249.0   
81                          0.95                   9.0         180.0   

     covid_patients_share               region  
119                  22.0  Southern California  
60                   10.0  Southern California  
138                   3.0  Southern California  
143                   4.0  Southern California  
122                  19.0  Southern California  
144                   8.0  Southern California  
64                    9.0  Southern California  
120                   4.0  Southern California  
102                   8.0  Southern California  
81                    5.0  Southern California

### How many LA County hospitals (with Covid patients) are in our dataframe? 

In [48]:
len(current_la[(current_la["total_covid_icu_patients"] > 0)])

63

### How many have 'high' ICU occupancy rates? 

In [49]:
len(current_la[current_la["pct_occupied_adult_icu_beds"] > 0.90])

21

### Top 10 Bay Area hospotals by Covid share? 

In [50]:
current_ca[current_ca["county"].isin(bayarea)].sort_values(
    "covid_patients_share", ascending=False
).head(10)

hospital hospital_subtype  \
300               Kaiser Foundation Hospital-San Jose       Short Term   
290  Kaiser Foundation Hospital - South San Francisco       Short Term   
149                        Kaiser Foundation Hospital       Short Term   
7                Kaiser Foundation Hospital - Fremont       Short Term   
292                          San Mateo Medical Center       Short Term   
12           Kaiser Foundation Hospital - San Leandro       Short Term   
299               Regional Medical Center of San Jose       Short Term   
4       Kaiser Foundation Hospital - Oakland/Richmond       Short Term   
11                  Stanford Health Care - Valleycare       Short Term   
3                                 Eden Medical Center       Short Term   

        ccn       week       county fips  total_beds  \
300  050604 2021-03-19  Santa Clara  085       158.0   
290  050070 2021-03-19    San Mateo  081        88.0   
149  050510 2021-03-19        Marin  041        63.0   
7    050512 2021-03-19      Alameda  001        90.0   
292  050113 2021-03-19    San Mateo  081        46.0   
12   050777 2021-03-19      Alameda  001       180.0   
299  050125 2021-03-19  Santa Clara  085       237.0   
4    050075 2021-03-19      Alameda  001       313.0   
11   050283 2021-03-19      Alameda  001       257.0   
3    050488 2021-03-19      Alameda  001       150.0   

     total_staffed_adult_icu_beds  total_covid_icu_patients  \
300                          19.0                       9.0   
290                           7.0                       1.0   
149                           7.0                       3.0   
7                             8.0                       2.0   
292                           4.0                       1.0   
12                           14.0                       4.0   
299                          34.0                       5.0   
4                            19.0                       6.0   
11                           22.0                       3.0   
3                            24.0                       4.0   

     total_available_adult_icu_beds  pct_occupied_adult_icu_beds  \
300                             2.0                         0.89   
290                             1.0                         0.86   
149                             1.0                         0.86   
7                               1.0                         0.88   
292                             0.0                         1.00   
12                              1.0                         0.93   
299                             2.0                         0.94   
4                               1.0                         0.95   
11                              5.0                         0.77   
3                               3.0                         0.88   

     total_covid_patients  all_patients  covid_patients_share    region  
300                  25.0         135.0                  0.19  Bay Area  
290                  13.0          76.0                  0.17  Bay Area  
149                   8.0          55.0                  0.15  Bay Area  
7                    11.0          76.0                  0.14  Bay Area  
292                   4.0          31.0                  0.13  Bay Area  
12                   19.0         149.0                  0.13  Bay Area  
299                  19.0         154.0                  0.12  Bay Area  
4                    28.0         243.0                  0.12  Bay Area  
11                   10.0          87.0                  0.11  Bay Area  
3                    12.0         126.0                  0.10  Bay Area

---

## Get medical facilities' geographic data

In [51]:
# From HHS: https://maps3.arcgisonline.com/arcgis/rest/services/A-16/HHS_IOM_Health_Resources/MapServer

In [52]:
hospitals = gpd.read_file(
    "/Users/mhustiles/data/github/AGStoShapefile/backupdir/HHS_Hospital/HHS_Hospital_1608139617293.geojson"
)

In [53]:
medical_centers = gpd.read_file(
    "/Users/mhustiles/data/github/AGStoShapefile/backupdir/HHS_Medical_Center/HHS_Medical_Center_1608139616289.geojson"
)

### Clean up the column names so we can merge the facilities dataframes

In [54]:
facilities = [hospitals, medical_centers]

In [55]:
hospitals.rename(
    columns={
        "Name_new": "name",
        "Address_1": "address",
        "City": "city",
        "State_1": "state",
        "ZipCode": "zipcode",
        "PhoneNum": "phone",
        "County_Nam": "county",
        "Provider_N": "provider_id",
        "Hospital_T": "type",
        "Hospital_O": "operation",
        "Emergency_": "emergency",
    },
    inplace=True,
)

In [56]:
medical_centers.rename(
    columns={
        "Hospital_N": "name",
        "Address1": "address",
        "City_1": "city",
        "State_1": "state",
        "ZipCode": "zipcode",
        "PhoneNum": "phone",
        "County_Nam": "county",
        "Provider_N": "provider_id",
        "Hospital_T": "type",
        "Hospital_O": "operation",
        "Emergency_": "emergency",
    },
    inplace=True,
)

### Concatenate the different facility types into one dataframe, and also filter that just to CA

In [57]:
all_medical_geo = pd.concat(facilities)

In [58]:
all_medical_geo_ca = all_medical_geo[all_medical_geo["state"] == "CA"]

### We might be better off using HHS' own locations data with CCNs

In [59]:
locations = pd.read_csv(
    "input/hospital_locations.csv",
    dtype={"latitude": float, "longitude": float, "CCN": str},
)

In [60]:
locations.columns = (
    locations.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace("(", "", regex=False)
    .str.replace(")", "", regex=False)
    .str.replace("-", "_", regex=False)
)

### Even though it has the wrong location for White Memorial. Let's fix.

In [61]:
locations.loc[
    (locations.facility_name == "WHITE MEMORIAL MEDICAL CENTER"), "longitude"
] = -118.2176219

In [62]:
locations.loc[
    (locations.facility_name == "WHITE MEMORIAL MEDICAL CENTER"), "latitude"
] = 34.0493044

### Convert the lon/lat fields into a geodataframe

In [63]:
locations = gpd.GeoDataFrame(
    locations, geometry=gpd.points_from_xy(locations.longitude, locations.latitude)
)

### And then confine it to California

In [64]:
locationsca = gpd.GeoDataFrame(locations[locations["state"] == "CA"])

### Make the CCN string match the hospital capacity data

In [65]:
locationsca["ccn"] = locationsca["ccn"].str.zfill(6)

---

### Merge with our CA medical facilities

In [66]:
current_ca_geo = locationsca.merge(current_ca, left_on="ccn", right_on="ccn")

In [67]:
len(current_ca_geo)

348

In [68]:
current_ca_geo.rename(
    columns={
        "county_x": "county",
        "city_x": "city",
        "address_x": "address",
        "state_x": "state",
    },
    inplace=True,
)

In [69]:
# current_ca_geo.drop(['county_y', 'hospital_subtype', 'provider_id', 'ccn', 'emergency', 'address', 'OBJECTID', 'state', 'name', 'phone', 'zipcode'], axis=1, inplace=True)

---

### Filter to just LA County hospitals

In [70]:
current_la_geo = current_ca_geo[current_ca_geo["fips"] == "037"]

### How many facilities in California?

In [71]:
len(current_ca_geo)

348

### How many in L.A. County?

In [72]:
len(current_la_geo)

85

---

### Export hospital specific data for maps and tables

In [73]:
current_ca_geo.to_file("output/current_ca_geo.geojson", driver="GeoJSON")

In [74]:
current_ca_geo.to_csv("output/current_ca_geo.csv", index=False)

In [75]:
current_la_geo.to_file("output/current_la_geo.geojson", driver="GeoJSON")

In [76]:
current_la_geo.to_csv("output/current_la_geo.csv", index=False)

---

## Aggregate

### What's the average share of Covid patients in hospitals statewide?

In [77]:
state = (
    current_ca.groupby(["week"])
    .agg({"total_covid_patients": "sum", "all_patients": "sum"})
    .reset_index()
)

In [78]:
state["covid_patients_share"] = round(
    (state["total_covid_patients"] / state["all_patients"]), 2
)

In [79]:
state.head()

week  total_covid_patients  all_patients  covid_patients_share
0 2021-03-19                2950.0       43761.0                  0.07

### Covid patients by county

In [80]:
counties = (
    trimmed_timeseries.groupby(["county", "week", "region"])
    .agg({"total_covid_patients": "sum", "all_patients": "sum"})
    .reset_index()
)

In [81]:
counties["covid_patients_share"] = round(
    (counties["total_covid_patients"] / counties["all_patients"]), 2
)

#### Which counties have the most Covid patients (in most recent week)?

In [82]:
counties[counties["week"] == counties["week"].max()].sort_values(
    "total_covid_patients", ascending=False
).head(5)

county       week               region  total_covid_patients  \
607      Los Angeles 2021-03-19  Southern California                 864.0   
1218       San Diego 2021-03-19  Southern California                 248.0   
981           Orange 2021-03-19  Southern California                 182.0   
1184  San Bernardino 2021-03-19  Southern California                 178.0   
1083       Riverside 2021-03-19  Southern California                 169.0   

      all_patients  covid_patients_share  
607        12934.0                  0.07  
1218        3867.0                  0.06  
981         3442.0                  0.05  
1184        2439.0                  0.07  
1083        2109.0                  0.08

#### Which counties have the highest percentage of Covid patients (in most recent week)?

In [83]:
counties[counties["week"] == counties["week"].max()].sort_values(
    "total_covid_patients", ascending=False
).head(5)

county       week               region  total_covid_patients  \
607      Los Angeles 2021-03-19  Southern California                 864.0   
1218       San Diego 2021-03-19  Southern California                 248.0   
981           Orange 2021-03-19  Southern California                 182.0   
1184  San Bernardino 2021-03-19  Southern California                 178.0   
1083       Riverside 2021-03-19  Southern California                 169.0   

      all_patients  covid_patients_share  
607        12934.0                  0.07  
1218        3867.0                  0.06  
981         3442.0                  0.05  
1184        2439.0                  0.07  
1083        2109.0                  0.08

### How has this changed over time? 

In [84]:
counties_grouped = (
    counties.groupby(["county", "week", "region"])
    .agg({"total_covid_patients": "sum", "covid_patients_share": "mean"})
    .reset_index()
)

In [85]:
la_grouped = pd.DataFrame(counties_grouped[counties_grouped["county"] == "Los Angeles"])

In [86]:
la_grouped.tail()

county       week               region  total_covid_patients  \
603  Los Angeles 2021-02-19  Southern California                2282.0   
604  Los Angeles 2021-02-26  Southern California                1639.0   
605  Los Angeles 2021-03-05  Southern California                1237.0   
606  Los Angeles 2021-03-12  Southern California                1035.0   
607  Los Angeles 2021-03-19  Southern California                 864.0   

     covid_patients_share  
603                  0.17  
604                  0.13  
605                  0.10  
606                  0.08  
607                  0.07

In [87]:
la_grouped.to_csv("output/la_grouped.csv", index=False)

In [88]:
alt.Chart(la_grouped).mark_area().encode(
    x=alt.X("week:T", axis=alt.Axis(format="", title=" ", tickCount=3)),
    y=alt.Y(
        "covid_patients_share",
        title="Covid patients in LA County hospitals",
        axis=alt.Axis(format="%", tickCount=6),
    ),
).properties(width=500, height=500)

alt.Chart(...)

### Just southern California counties? 

In [89]:
socal_grouped = (
    counties_grouped[counties_grouped["county"].isin(socal)]
    .groupby(["county", "week"])
    .agg({"total_covid_patients": "sum", "covid_patients_share": "mean"})
    .reset_index()
)

In [90]:
alt.Chart(socal_grouped).mark_area().encode(
    x=alt.X("week:T", axis=alt.Axis(format="", title=" ", tickCount=3)),
    y=alt.Y(
        "covid_patients_share",
        title=" ",
        axis=alt.Axis(format="%", title="", tickCount=6),
    ),
    facet=alt.Facet("county"),
).properties(
    width=200, height=200, title="Share of all patients with Covid in SoCal Counties"
)

alt.Chart(...)

### Bay Area? 

In [91]:
bayarea_grouped = (
    counties_grouped[counties_grouped["county"].isin(bayarea)]
    .groupby(["week"])
    .agg({"total_covid_patients": "sum", "covid_patients_share": "mean"})
    .reset_index()
)

In [92]:
bayarea_grouped.tail()

week  total_covid_patients  covid_patients_share
29 2021-02-19                 834.0              0.104444
30 2021-02-26                 696.0              0.085556
31 2021-03-05                 561.0              0.072222
32 2021-03-12                 506.0              0.062222
33 2021-03-19                 429.0              0.052222

In [93]:
alt.Chart(bayarea_grouped).mark_area().encode(
    x=alt.X("week:T", axis=alt.Axis(format="", title=" ", tickCount=3)),
    y=alt.Y(
        "covid_patients_share",
        title=" ",
        axis=alt.Axis(
            format="%",
            title="Share of all patients with Covid in Bay Area",
            tickCount=6,
        ),
    ),
).properties(width=500, height=500)

alt.Chart(...)

---

### What's going on in the most-populous counties 

In [94]:
counties_pop = pd.read_csv(
    "../census/processed/acs5_2018_population_counties.csv",
    dtype={"geoid": str, "population": int, "state": str, "county": str},
)

In [95]:
counties_pop.rename(columns={"universe": "population"}, inplace=True)

In [96]:
counties_pop.drop(
    ["universe_annotation", "universe_moe", "universe_moe_annotation"],
    axis=1,
    inplace=True,
)

In [97]:
ca_counties_pop = counties_pop[counties_pop["state"] == "06"]

In [98]:
big_ca_counties_pop = ca_counties_pop.sort_values("population", ascending=False).head(
    10
)

In [99]:
big_ca_counties_pop["name"] = big_ca_counties_pop["name"].str.replace(
    " County, California", ""
)

In [100]:
big_counties = list(big_ca_counties_pop.name)

In [101]:
big_counties_grouped = (
    counties_grouped[counties_grouped["county"].isin(big_counties)]
    .groupby(["county", "week", "region"])
    .agg({"total_covid_patients": "sum", "covid_patients_share": "mean"})
    .reset_index()
)

In [102]:
chart = (
    alt.Chart(big_counties_grouped)
    .mark_area()
    .encode(
        x=alt.X(
            "week:T", axis=alt.Axis(grid=False, format="%b.", title=" ", tickCount=4)
        ),
        y=alt.Y(
            "covid_patients_share",
            title=" ",
            axis=alt.Axis(
                format="%",
                title="",
                tickCount=5,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
            ),
        ),
        facet=alt.Facet(
            "county",
            columns=5,
            title=" ",
            sort=alt.EncodingSortField(
                "covid_patients_share", op="max", order="descending"
            ),
        ),
        color=alt.Color("region", legend=alt.Legend(orient="top", title=" ")),
    )
    .properties(
        width=200,
        height=200,
        title="Share of all hospital patients with COVID-19 in large California counties",
    )
)

chart.configure_view(strokeOpacity=0)

alt.Chart(...)

---

### Regions

In [103]:
regions_timeseries = (
    trimmed_timeseries.groupby(["week", "region"])
    .agg(
        {
            "total_covid_patients": "sum",
            "all_patients": "sum",
            "total_beds": "sum",
            "total_staffed_adult_icu_beds": sum,
        }
    )
    .reset_index()
)

In [104]:
regions_timeseries["covid_patients_share"] = round(
    (regions_timeseries["total_covid_patients"] / regions_timeseries["all_patients"]), 2
)

In [105]:
regions_timeseries.tail(5)

week               region  total_covid_patients  all_patients  \
165 2021-03-19             Bay Area                 460.0        8688.0   
166 2021-03-19   Greater Sacramento                 172.0        3483.0   
167 2021-03-19  Northern California                  26.0         719.0   
168 2021-03-19   San Joaquin Valley                 529.0        4630.0   
169 2021-03-19  Southern California                1763.0       26241.0   

     total_beds  total_staffed_adult_icu_beds  covid_patients_share  
165     13143.0                        1327.0                  0.05  
166      4599.0                         492.0                  0.05  
167      1322.0                         128.0                  0.04  
168      6294.0                         638.0                  0.11  
169     40384.0                        4425.0                  0.07

In [106]:
regions_timeseries[
    regions_timeseries["week"] == "2021-01-01"
].covid_patients_share.mean()

0.32

In [107]:
chart = (
    alt.Chart(regions_timeseries)
    .mark_area()
    .encode(
        x=alt.X(
            "week:T", axis=alt.Axis(grid=False, format="%b.", title=" ", tickCount=4)
        ),
        y=alt.Y(
            "covid_patients_share",
            title=" ",
            axis=alt.Axis(
                format="%",
                title="",
                tickCount=5,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
            ),
        ),
        facet=alt.Facet(
            "region",
            columns=5,
            title=" ",
            sort=alt.EncodingSortField(
                "covid_patients_share", op="max", order="descending"
            ),
        ),
        #     color=alt.Color('region', legend=alt.Legend(orient="top", title=' '))
    )
    .properties(
        width=200,
        height=200,
        title="Share of all hospital patients with COVID-19 by region",
    )
)

chart.configure_view(strokeOpacity=0)

alt.Chart(...)

---

### Hospital overcapacity

In [108]:
before = pd.read_csv("input/hospital_utilization.csv", dtype={"fac_no": str})

In [109]:
before[before["fac_name"].str.contains("KING")]

fac_no                                    fac_name     fac_city  \
185  106191230  MARTIN LUTHER KING, JR. COMMUNITY HOSPITAL  LOS ANGELES   

          county license_no fac_zip  icu_beds  tot_lic_beds  
185  Los Angeles  060000132   90059        20           131

In [110]:
current_ca[current_ca["ccn"] == "050779"]

hospital hospital_subtype     ccn  \
139  Martin Luther King, Jr. Community Hospital       Short Term  050779   

          week       county fips  total_beds  total_staffed_adult_icu_beds  \
139 2021-03-19  Los Angeles  037       184.0                          20.0   

     total_covid_icu_patients  total_available_adult_icu_beds  \
139                       1.0                            14.0   

     pct_occupied_adult_icu_beds  total_covid_patients  all_patients  \
139                          0.3                  19.0         109.0   

     covid_patients_share               region  
139                  0.17  Southern California

In [111]:
before[["fac_name"]].sort_values("fac_name", ascending=False).to_csv(
    "output/before_names.csv", index=False
)

In [112]:
current_ca[["hospital"]].sort_values("hospital", ascending=False).to_csv(
    "output/current_names.csv", index=False
)